Libraries Used

In [3]:
import tensorflow as tf
import numpy as np
import json as js
import os
from keras.models import Model #type: ignore
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Reshape #type: ignore
from keras.optimizers import Adam #type: ignore
from keras.utils import Sequence #type: ignore
from PIL import Image

Loading Paths

In [4]:
with open("config.json",'r') as file:
    paths = js.load(file)

In [5]:
def build_object_detector():
    inputs = Input(shape=(256, 256, 3))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(80, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(), loss='mse')
    return model

def load_json(json_path):
    with open(json_path, 'r') as f:
        return js.load(f)

class ObjectDetectionDataGenerator(Sequence):
    def __init__(self, json_path, image_dir, batch_size=16):
        self.bbox_data = load_json(json_path)
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.image_names = list(self.bbox_data.keys())
    
    def __len__(self):
        return int(np.floor(len(self.image_names) / self.batch_size))
    
    def __getitem__(self, index):
        batch_images = self.image_names[index * self.batch_size:(index + 1) * self.batch_size]
        images, labels = [], []
        for img_name in batch_images:
            img_path = os.path.join(self.image_dir, img_name)
            bbox = self.bbox_data[img_name]
            image = Image.open(img_path).convert('RGB').resize((224, 224))
            images.append(np.array(image) / 255.0)
            labels.append(np.array(bbox) / 224.0)
        return np.array(images), np.array(labels)

def train_object_detector(model, json_path, image_dir, epochs=10, batch_size=16):
    train_generator = ObjectDetectionDataGenerator(json_path, image_dir, batch_size)
    model.fit(train_generator, epochs=epochs)
    return model

model = build_object_detector()
trained_model = train_object_detector(model, paths["Preprocessed_Train_json"], paths["Train_resized"])


I0000 00:00:1743849356.910085    6113 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1743849357.326061    6113 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1743849357.334859    6113 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1743849357.342698    6113 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

AttributeError: 'list' object has no attribute 'keys'